In [5]:
import pandas as pd
from pathlib import Path

# --- Load data ---
LABKA_PATH = Path(r"C:\Users\kfq6\Documents\Data\LABKA.xlsx")
df = pd.read_excel(LABKA_PATH, engine="openpyxl")

# --- Keep only relevant columns (Over70 is gone) ---
cols = [
    "DW_EK_Borger",
    "Dato_Proevetagningstid",
    "Klok_Proevetagningstid",
    "Alder_Proevetagningstid",
    "Analysenavn",
    "Svar",
    "Enhed",
]
df = df[[c for c in cols if c in df.columns]].copy()

# --- Parse date/time (no other text meddling), add Testdato ---
df["Dato_Proevetagningstid"] = pd.to_datetime(df["Dato_Proevetagningstid"], errors="coerce")
df["Testdato"] = df["Dato_Proevetagningstid"].dt.date

# --- Build column header: "Analysenavn [Enhed]" (unit optional) ---
def make_col_name(row):
    analy = str(row.get("Analysenavn", ""))
    unit = row.get("Enhed")
    unit = "" if pd.isna(unit) else str(unit)
    return f"{analy} [{unit}]" if unit else analy

df["AnalyseKolonne"] = df.apply(make_col_name, axis=1)

# --- Keep only last entry per test per day (so duplicates collapse) ---
df = df.sort_values(["DW_EK_Borger", "Testdato", "AnalyseKolonne", "Dato_Proevetagningstid"])
df_last = df.groupby(["DW_EK_Borger", "Testdato", "AnalyseKolonne"], as_index=False).tail(1)

# --- Pivot: one column per "Analysenavn [Enhed]", values are raw Svar (unaltered) ---
wide = df_last.pivot(
    index=["DW_EK_Borger", "Testdato"],
    columns="AnalyseKolonne",
    values="Svar"
)

# --- Add metadata back (except Over70) ---
meta_cols = [c for c in ["Alder_Proevetagningstid", "Dato_Proevetagningstid", "Klok_Proevetagningstid"] if c in df.columns]
meta = (
    df.sort_values(["DW_EK_Borger", "Testdato", "Dato_Proevetagningstid"])
      .groupby(["DW_EK_Borger", "Testdato"], as_index=False)[meta_cols]
      .first()
)

wide = meta.merge(wide.reset_index(), on=["DW_EK_Borger", "Testdato"], how="left")

# --- Excel-safe headers (don’t touch content) ---
safe_cols = []
for c in wide.columns:
    c = str(c).replace("\n", " ").replace("/", "_").replace("  ", " ").strip()
    c = c.replace(" ", "_")
    safe_cols.append(c)
wide.columns = safe_cols




c:\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Wide table saved to: C:\Users\kfq6\Documents\Data\LABKA_wide_rawSvar.xlsx
Shape: (57480, 38)


In [9]:
# --- Drop all columns without units (but keep metadata columns) ---
meta_cols = ["DW_EK_Borger", "Testdato", "Alder_Proevetagningstid", "Dato_Proevetagningstid", "Klok_Proevetagningstid"]
unitless_cols = [c for c in wide.columns if "_[" not in c and c not in meta_cols]

print(f"Dropping {len(unitless_cols)} unitless columns:")
print(unitless_cols)

wide = wide.drop(columns=unitless_cols)


Dropping 16 unitless columns:
['B-Hæmoglobin', 'Hb_x0028_B_x0029_-Hæmoglobin_x0020_A1c_x0020__x0028_IFCC_x0029_', 'P-25-Hydroxy-Vitamin_x0020_D_x0028_D3_x002B_D2_x0029_', 'P-Albumin', 'P-Calcium', 'P-Calcium_x0020__x0028_albuminkorrigeret_x0029_', 'P-Kalium', 'P-Kolesterol', 'P-Kolesterol_x0020_HDL', 'P-Kolesterol_x0020_LDL', 'P-Kreatinin', 'P-Natrium', 'P-Triglycerid', 'P-Vitamin_x0020_B12', 'U-Albumin_x0020__x002F__x0020_Kreatinin-ratio', 'eGFR_x0020__x002F__x0020_1_x002C_73m_x00B2__x0028_CKD-EPI_x0029_']


In [10]:
# --- Save ---
out_path = Path(r"C:\Users\kfq6\Documents\Data\LABKA_wide_rawSvar.xlsx")
wide.to_excel(out_path, index=False)

print("Wide table saved to:", out_path)
print("Shape:", wide.shape)

Wide table saved to: C:\Users\kfq6\Documents\Data\LABKA_wide_rawSvar.xlsx
Shape: (57480, 22)
